In [1]:
from src.rag.components.retriever import HybridRetriever

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [3]:
from pathlib import Path

In [4]:
model_path = Path.cwd().joinpath("models")

In [5]:
embedding_model_path = model_path.joinpath(embedding_model_id ).__str__()

In [6]:
cross_encoder_kwargs = {
    "model_name": embedding_model_path,
    "trust_remote_code": True,
    "local_files_only": True,
    "config_kwargs": {"use_memory_efficient_attention": False, "unpad_inputs": False},
    "device": "cpu"
}

In [7]:

transformer_kwargs  = { "model_name_or_path": embedding_model_path,
    "trust_remote_code": True,
    "device": "cpu",
    "config_kwargs": {"use_memory_efficient_attention": False,
                   "unpad_inputs": False},
    "cache_folder": model_path}

In [8]:
spacy_model_id = "en_core_web_sm"

In [9]:
retriever = HybridRetriever(
    cross_encoder_kwargs=cross_encoder_kwargs,
    spacy_model=spacy_model_id,
    sentence_transformer_kwargs=transformer_kwargs,
    language="english"
)

Some weights of the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of NewForSequenceClassification were not initialized from the model checkpoint at /Users/esp.py/Projects/Personal/end-to-end-rag/models/dunzhang/stella_en_400M_v5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [27]:
query = "How do you work out your LTV ratio?"

In [28]:
semantic_search_results = retriever.semantic_search(query=query, limit=10)

In [29]:
keywords = retriever.perform_keyword_extraction(query)

In [30]:
keywords

'"LTV ratio"'

In [31]:
keywords_search_results = retriever.keyword_search(keywords=keywords, limit=10)

We can add a reranker but we decided to keep it simple for now.

In [32]:
for item in semantic_search_results:
    print(item.content)
    print(10 * "-")

Loan to value ratio, or LTV, is the ratio of what you borrow as a mortgage against how much you pay as a deposit. Here’s how loan to value ratio works: You pay a deposit of £20,000 for a property worth £200,000. You get a mortgage of £180,000 to pay for the rest. Your deposit covers 10% of the house price. So, your LTV is 90%.
----------
Loan to Value ratio is the percentage of borrowing you take out against the value of your home. Find out how it works here.
----------
Your lender will need to carry out an independent valuation of the property you want to buy. This is to make sure the house is worth what you’re offering to pay for it, and this will be used to work out your loan to value ratio.
----------
Learn what loan to value is and how it could affect you buying a home.
----------
Most lenders consider anything under 80% to be a good LTV ratio but will vary by lender. While it’s sometimes possible to borrow extra, anything above 80% tends to cost more. If you can, increase your de

For some reason the encoder is taking long to return.

In [33]:
for item in keywords_search_results:
    print(item.content)
    print(10 * "-")

A lender will want to know how much you’ve saved for a deposit. And they’ll also look at your loan to value (LTV) ratio. This is the amount of the property value you’ll need to borrow with a mortgage – usually expressed as a percentage.  The more you have saved and the better LTV ratio you have, the better chance you'll have of being accepted for a mortgage.
----------
There’s no sure-fire way to guarantee that you’re accepted for a mortgage. But there are things you can do to help increase your chances. Have a high deposit or low loan to value (LTV) ratio. The higher your deposit and lower your LTV ratio, the less money you’ll need to borrow. This can make lenders feel more relaxed about your ability to meet your payments. Build up your credit score. Building up your credit score illustrates that you can meet your repayments and manage your money. This could make companies more willing to lend you money. Learn how to improve your score. Having a strong financial history. If you have l

In [34]:
# scores = retriever.cross_encoder.predict(
#    [(query, item.content) for item in keywords_search_results + semantic_search_results])

### Generator

In [35]:
from src.rag.components.generator import  LLamaCppGeneratorComponent

In [36]:
prompt = "you are a helpful mortgage advisor"

In [37]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [38]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [39]:
llama_cpp_generator._ping_api()

True

In [40]:
documents = [doc.content for doc in keywords_search_results + semantic_search_results]

In [41]:
documents

["A lender will want to know how much you’ve saved for a deposit. And they’ll also look at your loan to value (LTV) ratio. This is the amount of the property value you’ll need to borrow with a mortgage – usually expressed as a percentage.\xa0 The more you have saved and the better LTV ratio you have, the better chance you'll have of being accepted for a mortgage.",
 'There’s no sure-fire way to guarantee that you’re accepted for a mortgage. But there are things you can do to help increase your chances. Have a high deposit or low loan to value (LTV) ratio.\xa0The higher your deposit and lower your LTV ratio, the less money you’ll need to borrow. This can make lenders feel more relaxed about your ability to meet your payments. Build up your credit score.\xa0Building up your credit score illustrates that you can meet your repayments and manage your money. This could make companies more willing to lend you money.\xa0Learn how to improve your score. Having a strong financial history. If you

In [42]:
llama_cpp_generator.generate_chat_input(query, documents)

[{'role': 'system', 'content': 'you are a helpful mortgage advisor'},
 {'role': 'user',
  'content': "\n            DOCUMENTS:\n            \n             - A lender will want to know how much you’ve saved for a deposit. And they’ll also look at your loan to value (LTV) ratio. This is the amount of the property value you’ll need to borrow with a mortgage – usually expressed as a percentage.\xa0 The more you have saved and the better LTV ratio you have, the better chance you'll have of being accepted for a mortgage. \n\n            \n             - There’s no sure-fire way to guarantee that you’re accepted for a mortgage. But there are things you can do to help increase your chances. Have a high deposit or low loan to value (LTV) ratio.\xa0The higher your deposit and lower your LTV ratio, the less money you’ll need to borrow. This can make lenders feel more relaxed about your ability to meet your payments. Build up your credit score.\xa0Building up your credit score illustrates that you

In [43]:
response = llama_cpp_generator.run(query, documents)

In [44]:
print(response)

To work out your Loan to Value (LTV) ratio, you need to divide the amount of money you will borrow from a mortgage against the value of the property. For example, if you pay a deposit of £20,000 for a property worth £200,000, you get a mortgage of £180,000 to pay for the rest. Your deposit covers 10% of the house price, so your LTV is 90%.
